In [2]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv
from datetime import datetime, timedelta

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [15]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location:str, unit:str="fahrenheit")->str:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "AirBlue",
        "flight": "AB643",
    }

    return json.dumps(flight_info)    

In [16]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messaging = [{"role": "user", "content": main_request}]
    tooling = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        },
        {
        "type": "function",
        "function": {
            "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
        }
    }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messaging,
        tools=tooling,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))

    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
            "get_flight_info": get_flight_info
        }  # only one function in this example, but you can have multiple
        
        messaging.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
               **function_args,  # Pass all arguments as keyword arguments
            )
            messaging.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messaging))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messaging,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [17]:
run_conversation("When is the next flight from Paris to London?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_hvVnmxAI7HwBoUCIw3rAVxhQ', function=Function(arguments='{"loc_origin": "CDG", "loc_destination": "LHR"}', name='get_flight_info'), type='function'),
  ChatCompletionMessageToolCall(id='call_iNp1l1EKyRHs84nfm4mV5wqO', function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_hvVnmxAI7HwBoUCIw3rAVxhQ', function=Function(arguments='{"loc_origin": "CDG", "loc_destination": "LHR"}', name='get_flight_info'), type='function'),
 ChatCompletionMessageToolCall(id='call_iNp1l1EKyRHs84nfm4mV5wqO', function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user', 'content': 'When is the next flight from Paris to London?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hvVnmxAI7HwBoUCIw3rAVxhQ', function=Function(arguments='{"loc_origin": "CDG", "loc_destination": "LHR"}', name='get_flight_info'), type='function'), ChatCompletionMessageToolCall(id='call_iNp1l1EKyRHs84nfm4mV5wqO', function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_hvVnmxAI7HwBoUCIw3rAVxhQ',
  'role': 'tool',
  'name': 'get_flight_info',
  'content': '{"loc_origin": "CDG", "loc_destination": "LHR", "datetime": "2023-12-03 23:12:38.864744", "airline": "AirBlue", "flight": "AB643"}'},
 {'tool_call_id': 'call_iNp1l1EKyRHs84nfm4mV5wqO',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Paris", "temperature": "22", "unit": "celsius"}'}]

* Second Response:  {'id': 'chatcmpl-8RjN9ujzvvS5NnkOrqcCmMnYJ1XtV', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The next flight from Paris (CDG) to London (LHR) is with AirBlue flight AB643, departing on December 3, 2023.', role='assistant', function_call=None, tool_calls=None))], 'created': 1701619959, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_eeff13170a', 'usage': CompletionUsage(completion_tokens=33, prompt_tokens=154, total_tokens=187)}


'The next flight from Paris (CDG) to London (LHR) is with AirBlue flight AB643, departing on December 3, 2023.'